In [22]:
import csv
from collections import deque
from tabulate import tabulate

# read in from csv file
def parse_file(file_name):
    with open(file_name, 'r') as file:
        # read lines from csv file
        lines = list(csv.reader(file))

        # header parsing
        machine_name = lines[0][0]
        start_state = lines[4][0]
        accept_state = lines[5][0]
        reject_state = lines[6][0]

        # parse transitions
        transitions = {}
        for line in lines[7:]:

            # split lines by commas
            if len(line) == 1:
                line = line[0].split(',')

            # ignore lines with not enough elements
            if not line or len(line) < 5:
                continue

            # split line into parts
            state, read_char, new_state, write_char, direction = line
            # use current state and read character as key
            key = (state, read_char)
            if key not in transitions:
                # initialize list
                transitions[key] = []
            # add the transition
            transitions[key].append((new_state, write_char, direction))
    return machine_name, start_state, accept_state, reject_state, transitions

# simulate ntm with bfs
def simulate_ntm(input_string, transitions, start_state, accept_state, reject_state, max_depth=100):
    # initial configuration (left tape, current state, right tape)
    initial_config = ("", start_state, input_string)
    queue = deque([initial_config])
    # reconstruction path of computation
    parent_map = {initial_config: None}
    configurations_per_level = [1]
    total_transitions = 0
    # flag if all configurations reject
    all_configs_rejected = True
    max_transitions = 1000

    # bfs loop
    while queue and total_transitions < max_depth:
        current_level_size = len(queue)
        next_level_configs = 0

        for _ in range(current_level_size):
            left, state, right = queue.popleft()

            # check for accept and reject states
            if state == accept_state:
                # calculate nondeterminism
                all_configs_rejected = False
                avg_nondeterminism = sum(configurations_per_level) / len(configurations_per_level)
                return "accept", trace_path(parent_map, (left, state, right)), total_transitions, avg_nondeterminism

            # determine current head character
            head_char = right[0] if right else '_'
            key = (state, head_char)

            # explore possible transitions
            if key in transitions:
                for new_state, write_char, direction in transitions[key]:
                    # update tape and create new configuration
                    new_left, new_right = move_tape(left, right, write_char, direction)
                    new_config = (new_left, new_state, new_right)

                    # avoid same configuration
                    if new_config not in parent_map:
                        queue.append(new_config)
                        parent_map[new_config] = (left, state, right)
                        next_level_configs += 1

            # increment transition counter
            total_transitions += 1

            if total_transitions >= max_transitions:
                return "stopped", [], total_transitions, 0

        # update nondeterminism
        configurations_per_level.append(next_level_configs)

        # stop if no more paths to explore
        if not queue:
          break

    avg_nondeterminism = sum(configurations_per_level) / len(configurations_per_level)
    return "reject", [], total_transitions, avg_nondeterminism

# move tape and update configuration
def move_tape(left, right, write_char, direction):
    if direction == "R":
      # move right, add write_char to left and remove first char from right
        new_left = left + write_char
        new_right = right[1:] if len(right) > 1 else "_"
    elif direction == "L":
      # move left, remove last char from left,
        new_left = left[:-1] if left else "_"
        new_right = (left[-1] if left else "_") + write_char + right
    else:
        raise ValueError("Invalid move direction")
    return new_left, new_right

# trace path back
def trace_path(parent_map, config):
    path = []
    while config is not None:
        # add current configuration to path
        path.append(config)
        config = parent_map.get(config, None)
    # reverse path to start
    return list(reversed(path))

def main():
    # Parse machine file
    machine_details = parse_file("aplus.csv")
    if not machine_details:
        return

    # get machine details
    machine_name, start_state, accept_state, reject_state, transitions = machine_details

    # test input strings
    test_strings = ["aaaaaaaaa", "a", "b", "aaaaa", "bbbb", "aaabaaa"]
    results = []

    for input_string in test_strings:
        # simulate turing machine
        result, path, transitions_count, nondeterminism = simulate_ntm(
            input_string, transitions, start_state, accept_state, reject_state
        )
        # calculate tree depth
        tree_depth = len(path) - 1 if path else 0
        # create acceptance path
        acceptance_path = "\n".join(
            [f"{config[0]} | {config[1]} | {config[2]}" for config in path]
            ) if path else "None"

        if result == "stopped":
          execution_summary = f"Execution stopped after {transitions_count} \ntransitions (step limit exceeded)."
        elif result == "accept" and path:
            execution_summary = f"String accepted in {tree_depth} \ntransitions on the accepting path."
        else:
            execution_summary = f"String rejected in {tree_depth} \ntransitions (max depth explored)."

        # add new results to results list
        results.append([
            machine_name,
            input_string,
            result,
            tree_depth,
            transitions_count,
            f"{nondeterminism:.2f}",
            acceptance_path,
            execution_summary
        ])

    # generate the table
    headers = [
    "Machine Name", "String", "Result", "Depth",
    "Configurations", "Nondeterminism", "Acceptance Path", "Execution Summary"
    ]
    # print the table
    print("\nSimulation Results:\n")
    print(tabulate(results, headers=headers, tablefmt="grid"))
if __name__ == "__main__":
    main()


Simulation Results:

+----------------+-----------+----------+---------+------------------+------------------+---------------------+------------------------------------+
| Machine Name   | String    | Result   |   Depth |   Configurations |   Nondeterminism | Acceptance Path     | Execution Summary                  |
+================+===========+==========+=========+==================+==================+=====================+====================================+
| ﻿a plus         | aaaaaaaaa | accept   |      10 |               19 |             1.82 | | q1 | aaaaaaaaa    | String accepted in 10              |
|                |           |          |         |                  |                  | a | q1 | aaaaaaaa   | transitions on the accepting path. |
|                |           |          |         |                  |                  | aa | q1 | aaaaaaa   |                                    |
|                |           |          |         |                  |             